# Optical Flow

Submitted by: Ephi Frankel


In [15]:
%matplotlib inline

import cv2
import numpy as np


###  Sparse optical flow

In this part you will calculate and visualize optical flow on a live video feed from a webcam.

First, find some good features to track, use either Harris corners, or Shi-Tomasi corners (`cv2.goodFeaturesToTrack`).

Then, you will track these features from frame to frame, and display their location on the live video feed. Use: `cv2.calcOpticalFlowPyrLK`.

As features tend to get lost during the time (check their returned status), it is advised to look for new features to track, from time to time (either every N frames, or when the features number falls below some level).

Try to add a visual trace of the motion. See for example: https://www.youtube.com/watch?v=E86NLzNbuL8


- API doc: https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html

- Example: http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html



In [2]:
# your code here...
cap = cv2.VideoCapture(0)
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 500,
                       qualityLevel = 0.01,
                       minDistance = 1,
                       blockSize = 3)

# Parameters for lucas kanade optial flow
lk_params = dict( winSize  = (13,13),
                  maxLevel = 7,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 20, 0.01))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
update = 0
while(1):
    mask = np.zeros_like(old_frame)
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)    
    # calculate optical flow
 
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = np.array(p1[st==1])
    good_old = np.array(p0[st==1])
    dist_arr = np.sqrt((good_new[:,0]-good_old[:,0])**2 + (good_new[:,1]-good_old[:,1])**2).astype('int')
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        #ang_new = np.round(np.arctan2(b,a)*180/np.pi)
        #ang_old = np.round(np.arctan2(d,c)*180/np.pi)
        #prox =  np.abs(a-c) + np.abs(c-d)
        clr =[255,0,0]
        if(dist_arr[i] < 80):
            if(dist_arr[i] > 5):
                clr =[255,0,50]
            if(dist_arr[i] > 20):
                clr =[255,0,200]
            if(dist_arr[i] > 50):
                clr =[255,0,200]
            mask = cv2.line(mask, (a,b),(c,d), clr, 2)
        cv2.circle(frame,(a,b),5,clr,-1) 
      
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    #old_gray = cv2.pyrDown(old_gray)
    if(update % 100 == 0 or len(good_new) < 100):
        p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
        update = 0
    else:
        p0 = good_new.reshape(-1,1,2)
    update = update + 1

cv2.destroyAllWindows()
cap.release()


### Interactive tracking

In this part, we will add some interactive aspects to your previous implementation. 

First, the live video is shown without any features. 

Then, the user draws a region-of-interest (ROI) box on the video. Only the features that fall inside this region should be tracked from frame to frame. 

In every frame, plot the bounding box of the tracked features (using cv2.boundingRect).

Once again, since features tend to get lost over time, find some new features to track every now and then.

See for example: https://www.youtube.com/watch?v=kcrGkD2HOZs

Tutorial on handling mouse events in OpenCV: 
- http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_mouse_handling/py_mouse_handling.html



In [ ]:
# your code here..
cap = cv2.VideoCapture(0)
cv2.namedWindow('frame2')
ret,frame = cap.read()
frame_rect = np.zeros_like(frame)

drawing = False # true if mouse is pressed
ix,iy = -1,-1
ex,ey = None,None
# mouse callback function
def draw_rect(event,x,y,flags,param):
    global ix,iy,drawing,mode,frame_rect,ex,ey
    frame_rect = np.zeros_like(frame)
    if event == cv2.EVENT_LBUTTONDOWN: 
        drawing = True
        ex,ey = None,None
        ix,iy = x,y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
                cv2.rectangle(frame_rect,(ix,iy),(x,y),(0,255,0),2)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(frame_rect,(ix,iy),(x,y),(0,255,0),2)
        ex = x
        ey = y
        frame_rect = np.zeros_like(frame)
        
def clean_p0(p0,sx,endx,sy,endy,margin = 0):
    delIndx = 0
    for p in p0:
        px = p[0][0]
        py = p[0][1]
        if(px < (sx-margin) or px > (endx+margin) or py < (sy-margin) or py > (endy+margin) ):
            p0 = np.delete(p0, delIndx, axis=0)
        else:
            delIndx = delIndx + 1
    return p0
cv2.setMouseCallback('frame2',draw_rect)
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 600,
                       qualityLevel = 0.0000001,
                       minDistance = 1,
                       blockSize = 5)

# Parameters for lucas kanade optial flow
lk_params = dict( winSize  = (13,13),
                  maxLevel = 11,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 20, 0.01))
old_gray = None
allowquit = True
p0 = None
distLock = None
while(1):
    ret,frame = cap.read()
    frame = cv2.add(frame,frame_rect)
    if(ex != None and ey != None):
        allowquit = False
        if(p0 is None):
            old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
            p0= clean_p0(p0,ix,ex,iy,ey)
            if(len(p0) < 1):
                ex,ey = None,None
        else:
            mask = np.zeros_like(frame)
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
            # Select good points
            good_new = np.array(p1[st==1])
            if(good_new is None or len(good_new) < 1):
                ex,ey = None,None
                continue
            good_old = np.array(p0[st==1])
            dist_arr = np.sqrt((good_new[:,0]-good_old[:,0])**2 + (good_new[:,1]-good_old[:,1])**2).astype('int')
            if(distLock is not None):
                fillter = dist_arr < distLock
                good_new = good_new[fillter]
                good_old = good_old[fillter]
                dist_arr = dist_arr[fillter]
                if(good_new is None or len(good_new) < 1):
                    ex,ey = None,None
                    continue
                ang_old = np.arctan2(good_old[:,1],good_old[:,0])*180/np.pi
                ang_new = np.arctan2(good_new [:,1],good_new [:,0])*180/np.pi
                ang_diff = np.abs(ang_new-ang_old)
                fillter_ang = ang_diff < 90
                good_new = good_new[fillter_ang]
                good_old = good_old[fillter_ang]
                if(good_new is None or len(good_new) < 1):
                    ex,ey = None,None
                    continue
            maxAll = np.amax(good_new, axis = 0)
            minAll = np.amin(good_new, axis = 0)
            maxX = maxAll[0]
            maxY = maxAll[1]
            minX = minAll[0]
            minY = minAll[1]
            if(distLock is None):
                distLock = np.sqrt( (maxX-minX)**2 + (maxY-minY)**2)*0.8
                print('dist lock is: ', distLock)
            
            
            # draw the tracks
            for i,(new,old) in enumerate(zip(good_new,good_old)):
                a,b = new.ravel()
                c,d = old.ravel()
                clr =[255,0,0]
                if(dist_arr[i] > 5):
                    clr =[255,0,50]
                    if(dist_arr[i] > 10):
                        clr =[255,0,80]
                    mask = cv2.line(mask, (a,b),(c,d), clr, 2)
                cv2.circle(frame,(a,b),5,clr,-1)
            frame= cv2.add(frame,mask)
            mx = np.int0(minX)
            my = np.int0(minY)
            mex = np.int0(maxX)
            mey = np.int0(maxY)
            cv2.rectangle(frame, (mx, my), (mex, mey), (0,255,0), 2)
            old_gray = frame_gray.copy()
            allowquit = True
            if(len(p0) < 1):
                p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
                p0= clean_p0(p0,mx,mex,my,mey,40)
                if(len(p0) < 1):
                    ex,ey = None,None
            else:
                p0 = good_new.reshape(-1,1,2)
    else:
        p0 = None
        allowquit = True
        distLock = None
    cv2.imshow('frame2',frame)
    if (cv2.waitKey(1) & 0xFF == ord('q')) and allowquit:
        break
cv2.destroyAllWindows()
cap.release()

## Good Luck!